## UPI Analyzer

In [1]:
from langchain_community.document_loaders import PyPDFLoader
loader = PyPDFLoader("737893745-PhonePe-Transaction-Statement-3.pdf")
docs = loader.load()
docs

[Document(metadata={'producer': 'iLovePDF', 'creator': 'PyPDF', 'creationdate': '', 'moddate': '2024-05-29T18:20:51+00:00', 'source': '737893745-PhonePe-Transaction-Statement-3.pdf', 'total_pages': 8, 'page': 0, 'page_label': '1'}, page_content='Transaction Statement for +919843968308\nApr 01, 2023 - Mar 31, 2024\nDate Transaction Details Type Amount\nApr 03, 2023\n12:34 PM\nReceived from global books\nTransaction ID : T2304031234001870744941\nUTR No : 309333675610\nCredited to XX3466\nCredit INR \n28600.00\nApr 09, 2023\n08:04 PM\nPaid to syam.ks\nTransaction ID : T2304092004042147133838\nUTR No : 309983907956\nPaid by XX3466\nDebit INR 3700.00\nApr 26, 2023\n10:23 AM\nReceived from global books\nTransaction ID : T2304261023295057948675\nUTR No : 311641997371\nCredited to XX3466\nCredit INR \n39600.00\nMay 02, 2023\n03:54 PM\nBill paid - Electricity\nTransaction ID : NX23050215542726535988111\nUTR No : 312248012348\nPaid by XX3466\nDebit INR 630.00\nJun 09, 2023\n07:48 PM\nReceived fr

In [2]:

from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=20)
text_splitter.split_documents(docs)[:5]

[Document(metadata={'producer': 'iLovePDF', 'creator': 'PyPDF', 'creationdate': '', 'moddate': '2024-05-29T18:20:51+00:00', 'source': '737893745-PhonePe-Transaction-Statement-3.pdf', 'total_pages': 8, 'page': 0, 'page_label': '1'}, page_content='Transaction Statement for +919843968308\nApr 01, 2023 - Mar 31, 2024\nDate Transaction Details Type Amount\nApr 03, 2023\n12:34 PM\nReceived from global books\nTransaction ID : T2304031234001870744941\nUTR No : 309333675610\nCredited to XX3466\nCredit INR \n28600.00\nApr 09, 2023\n08:04 PM\nPaid to syam.ks\nTransaction ID : T2304092004042147133838\nUTR No : 309983907956\nPaid by XX3466\nDebit INR 3700.00\nApr 26, 2023\n10:23 AM\nReceived from global books\nTransaction ID : T2304261023295057948675\nUTR No : 311641997371\nCredited to XX3466\nCredit INR \n39600.00\nMay 02, 2023\n03:54 PM\nBill paid - Electricity\nTransaction ID : NX23050215542726535988111\nUTR No : 312248012348\nPaid by XX3466\nDebit INR 630.00\nJun 09, 2023\n07:48 PM\nReceived fr

In [3]:

documents=text_splitter.split_documents(docs)
documents

[Document(metadata={'producer': 'iLovePDF', 'creator': 'PyPDF', 'creationdate': '', 'moddate': '2024-05-29T18:20:51+00:00', 'source': '737893745-PhonePe-Transaction-Statement-3.pdf', 'total_pages': 8, 'page': 0, 'page_label': '1'}, page_content='Transaction Statement for +919843968308\nApr 01, 2023 - Mar 31, 2024\nDate Transaction Details Type Amount\nApr 03, 2023\n12:34 PM\nReceived from global books\nTransaction ID : T2304031234001870744941\nUTR No : 309333675610\nCredited to XX3466\nCredit INR \n28600.00\nApr 09, 2023\n08:04 PM\nPaid to syam.ks\nTransaction ID : T2304092004042147133838\nUTR No : 309983907956\nPaid by XX3466\nDebit INR 3700.00\nApr 26, 2023\n10:23 AM\nReceived from global books\nTransaction ID : T2304261023295057948675\nUTR No : 311641997371\nCredited to XX3466\nCredit INR \n39600.00\nMay 02, 2023\n03:54 PM\nBill paid - Electricity\nTransaction ID : NX23050215542726535988111\nUTR No : 312248012348\nPaid by XX3466\nDebit INR 630.00\nJun 09, 2023\n07:48 PM\nReceived fr

In [4]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_community.vectorstores import FAISS
from dotenv import load_dotenv
import os
# Load API key from .env
load_dotenv()
os.environ["GOOGLE_API_KEY"] = os.getenv("GOOGLE_API_KEY")

# Initialize Gemini embeddings
embeddings = GoogleGenerativeAIEmbeddings(
    model="models/gemini-embedding-001"   # ✅ Gemini embedding model
)
db=FAISS.from_documents(documents,embeddings)



d:\Data Science\LangChain\GenAi_BotGemma\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
# using gemini
from langchain_google_genai import ChatGoogleGenerativeAI

import os
from dotenv import load_dotenv

# Load .env file
load_dotenv()

# ✅ Correct env variable
api_key = os.getenv("GOOGLE_API_KEY")
llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-pro",
    google_api_key=api_key,   
    temperature=0.7
)

In [6]:
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_template("""
Answer the following question based only on the provided conext
Think step by step before providing a detailed answer
I'll tip you $1000 if the user finds the answer helpful
<context>{context}</context>
Question: {input}

""")

In [7]:
## Chain
from langchain.chains.combine_documents import create_stuff_documents_chain

document_Chain=create_stuff_documents_chain(llm,prompt)

In [8]:
"""
Retrievers: A retriever is an interface that returns documents given
 an unstructured query. It is more general than a vector store.
 A retriever does not need to be able to store documents, only to 
 return (or retrieve) them. Vector stores can be used as the backbone
 of a retriever, but there are other types of retrievers as well. 
 https://python.langchain.com/docs/modules/data_connection/retrievers/   
"""

retriever=db.as_retriever()
retriever

VectorStoreRetriever(tags=['FAISS', 'GoogleGenerativeAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000001E6D4D7FE30>, search_kwargs={})

In [9]:
from langchain.chains import create_retrieval_chain
retriever_chain=create_retrieval_chain(retriever,document_Chain)

In [11]:
response=retriever_chain.invoke({"input":""" Total expense month wise"""})

In [13]:
response['answer']

'Based on the provided transaction statement, here is the month-wise breakdown of total expenses. Expenses are identified as "Debit" transactions in the statement.\n\n### Step-by-Step Calculation\n\n**Step 1: Identify all Debit Transactions**\nFirst, we will scan the document and list all transactions marked as "Debit" along with their date and amount.\n\n*   **Apr 09, 2023:** Paid to syam.ks - Debit INR 3,700.00\n*   **May 02, 2023:** Bill paid - Electricity - Debit INR 630.00\n*   **Jun 14, 2023:** Paid to ******9368 - Debit INR 3,300.00\n*   **Oct 29, 2023:** Paid to Ravichandran - Debit INR 3,000.00\n*   **Oct 31, 2023:** Bill paid - Electricity - Debit INR 536.00\n*   **Nov 16, 2023:** Paid to ******5339 - Debit INR 18,000.00\n*   **Nov 30, 2023:** Paid to ******5339 - Debit INR 30,000.00\n*   **Dec 01, 2023:** Paid to ******5339 - Debit INR 10,000.00\n*   **Dec 02, 2023:** Paid to ******5339 - Debit INR 10,000.00\n*   **Dec 26, 2023:** Paid to ******5339 - Debit INR 20,000.00\n* 